In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.models.segmentation import deeplabv3_mobilenet_v3_large
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

# 고급 데이터 증강 및 전처리 클래스
class AdvancedTransforms:
    def __init__(self):
        self.transform = transforms.Compose([
            transforms.RandomResizedCrop(256),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
            transforms.RandomRotation(15),
            transforms.GaussianBlur(3, sigma=(0.1, 2.0)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

    def __call__(self, image):
        image = self.transform(image)
        image_np = np.array(image)
        image_np = self.add_motion_blur(image_np)
        return transforms.ToTensor()(Image.fromarray(image_np))

    def add_motion_blur(self, image_np):
        size = 15
        kernel = np.zeros((size, size))
        kernel[int((size-1)/2), :] = np.ones(size)
        kernel = kernel / size
        image_blurred = cv2.filter2D(image_np, -1, kernel)
        return image_blurred

# 사용자 정의 데이터셋 클래스
class CustomDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx])
        
        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        mask = torch.from_numpy(np.array(mask)).long()
        
        return image, mask

# 데이터 경로 및 하이퍼파라미터 설정
image_dir = os.path.join('training', 'images')
mask_dir = os.path.join('training', 'labels')
batch_size = 8
num_classes = 25
num_epochs = 20
learning_rate = 0.001

# 데이터셋 및 데이터로더 생성
transform = AdvancedTransforms()
dataset = CustomDataset(image_dir, mask_dir, transform=transform)

# 데이터셋 분할
train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# 모델 정의 및 전이 학습
model = deeplabv3_mobilenet_v3_large(pretrained=True)  # 사전 훈련된 모델 사용
model.classifier[4] = nn.Conv2d(256, num_classes, kernel_size=(1, 1))  # 클래스 수에 맞게 조정

# 일부 레이어만 학습 가능하도록 설정
for param in model.parameters():
    param.requires_grad = False

# 마지막 레이어 학습 가능하도록 설정
for param in model.classifier.parameters():
    param.requires_grad = True

# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 모델 학습
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, masks in train_loader:
        images = images.to(device)
        masks = masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)['out']
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

# 모델 검증
model.eval()
val_loss = 0.0
with torch.no_grad():
    for images, masks in val_loader:
        images = images.to(device)
        masks = masks.to(device)
        outputs = model(images)['out']
        loss = criterion(outputs, masks)
        val_loss += loss.item()

    print(f"Validation Loss: {val_loss / len(val_loader):.4f}")

# 학습된 모델 저장
torch.save(model.state_dict(), 'finetuned_autonomous_driving_segmentation_model.pth')



FileNotFoundError: [Errno 2] No such file or directory: '/home/mira/Desktop/KistAIRobot/david/autonomous-driving/project/2DSS/training/labels/E_DCG_230829_302_FC_323.jpg'

In [ ]:
# 모델 로드
def load_model(model_path):
    model = deeplabv3_mobilenet_v3_large(pretrained=False)
    model.classifier[4] = nn.Conv2d(256, num_classes, kernel_size=(1, 1))  # 클래스 수에 맞게 조정
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

model = load_model('finetuned_autonomous_driving_segmentation_model.pth')


## test

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.models.segmentation import deeplabv3_mobilenet_v3_large
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import numpy as np
from sklearn.model_selection import train_test_split
import cv2

# 고급 데이터 증강 및 전처리 클래스
class AdvancedTransforms:
    def __init__(self):
        self.transform = transforms.Compose([
            transforms.RandomResizedCrop(256),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
            transforms.RandomRotation(15),
            transforms.GaussianBlur(3, sigma=(0.1, 2.0)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

    def __call__(self, image):
        image = self.transform(image)
        # Add motion blur to the image
        image_np = np.array(image)
        image_np = self.add_motion_blur(image_np)
        return transforms.ToTensor()(Image.fromarray(image_np))

    def add_motion_blur(self, image_np):
        size = 15
        kernel = np.zeros((size, size))
        kernel[int((size-1)/2), :] = np.ones(size)
        kernel = kernel / size
        image_blurred = cv2.filter2D(image_np, -1, kernel)
        return image_blurred

# 사용자 정의 데이터셋 클래스
class CustomDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx])
        
        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        if self.transform is not None:
            image = self.transform(image)
            mask = self.transform(mask)

        mask = torch.from_numpy(np.array(mask)).long()
        
        return image, mask

# 데이터 경로 및 하이퍼파라미터 설정
image_dir = "path/to/images"
mask_dir = "path/to/masks"
batch_size = 8
num_classes = 25
num_epochs = 20
learning_rate = 0.001

# 데이터셋 및 데이터로더 생성
transform = AdvancedTransforms()  # 고급 데이터 증강 적용

dataset = CustomDataset(image_dir, mask_dir, transform=transform)
train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# 모델 정의 및 저장된 가중치 불러오기
model = deeplabv3_mobilenet_v3_large(pretrained=False)
model.classifier[4] = nn.Conv2d(256, num_classes, kernel_size=(1, 1))  # 클래스 수에 맞게 조정

# 학습된 모델이 저장된 파일을 로드
model.load_state_dict(torch.load('best_autonomous_driving_segmentation_model.pth'), strict=False)

# 일부 레이어만 학습 가능하도록 설정
for param in model.parameters():
    param.requires_grad = False

# 필요한 레이어만 학습 가능하도록 설정 (예: 마지막 레이어)
for param in model.classifier.parameters():
    param.requires_grad = True

# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)

# 모델 학습
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, masks in train_loader:
        optimizer.zero_grad()
        outputs = model(images)['out']
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

# 모델 검증
model.eval()
val_loss = 0.0
with torch.no_grad():
    for images, masks in val_loader:
        outputs = model(images)['out']
        loss = criterion(outputs, masks)
        val_loss += loss.item()
    print(f"Validation Loss: {val_loss / len(val_loader):.4f}")

# 학습된 모델 저장
torch.save(model.state_dict(), f'finetuned_autonomous_driving_segmentation_model{3}.pth')


FileNotFoundError: [Errno 2] No such file or directory: 'path/to/images'

In [3]:
import os
import json
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.models.segmentation import deeplabv3_mobilenet_v3_large
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.cuda.amp import GradScaler, autocast
import torch.quantization
from torch.optim.lr_scheduler import StepLR

# 데이터셋 클래스 정의
class AutonomousDrivingDataset(Dataset):
    def __init__(self, root_dir, original_transform=None, augment_transform=None):
        self.root_dir = root_dir
        self.original_transform = original_transform
        self.augment_transform = augment_transform
        self.images_dir = os.path.join(root_dir, 'images')
        self.labels_dir = os.path.join(self.root_dir, 'labels')
        self.image_files = sorted(os.listdir(self.images_dir))

    def __len__(self):
        return len(self.image_files) * 2  # 원본 + 증강 데이터

    def __getitem__(self, idx):
        real_idx = idx // 2
        is_augmented = idx % 2 == 1

        img_name = self.image_files[real_idx]
        img_path = os.path.join(self.images_dir, img_name)
        label_path = os.path.join(self.labels_dir, img_name.replace('.jpg', '.json'))

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        with open(label_path, 'r') as f:
            label_data = json.load(f)

        annotations = sorted(label_data['Annotation'], key=lambda x: label_data['Annotation'].index(x), reverse=True)
        mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)

        class_mapping = {
            'road': 1, 'sidewalk': 2, 'road roughness': 3, 'road boundaries': 4, 'crosswalks': 5,
            'lane': 6, 'road color guide': 7, 'road marking': 8, 'parking': 9, 'traffic sign': 10,
            'traffic light': 11, 'pole/structural object': 12, 'building': 13, 'tunnel': 14,
            'bridge': 15, 'pedestrian': 16, 'vehicle': 17, 'bicycle': 18, 'motorcycle': 19,
            'personal mobility': 20, 'dynamic': 21, 'vegetation': 22, 'sky': 23, 'static': 24
        }

        for annotation in annotations:
            points = np.array(annotation['data'][0]).reshape((-1, 1, 2)).astype(np.int32)
            class_name = annotation['class_name']
            class_id = class_mapping.get(class_name, 0)
            cv2.fillPoly(mask, [points], class_id)

        # 데이터 증강 적용
        if is_augmented and self.augment_transform:
            augmented = self.augment_transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
        elif self.original_transform:
            augmented = self.original_transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask.long()

# 데이터 전처리 및 증강 정의
train_transform = A.Compose([
    A.Rotate(limit=15, p=0.5),  # 회전: -15도에서 15도까지
    A.HorizontalFlip(p=0.5),  # 수평 뒤집기
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),  # 밝기 조절
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),  # 가우시안 노이즈 추가
    A.OneOf([
        A.GaussianBlur(blur_limit=(1, 5), p=0.5),  # 가우시안 블러
        A.MedianBlur(blur_limit=3, p=0.5)  # 미디안 블러
    ], p=0.3), 
    A.OneOf([
        A.RandomShadow(num_shadows_lower=1, num_shadows_upper=3, shadow_dimension=5, shadow_roi=(0, 0.5, 1, 1), p=0.5),
        A.RandomFog(fog_coef_lower=0.3, fog_coef_upper=0.8, alpha_coef=0.1, p=0.5),
        A.RandomRain(slant_lower=-10, slant_upper=10, drop_length=20, drop_width=1, drop_color=(200, 200, 200), p=0.5),
    ], p=0.3),
    A.Resize(256, 256),  # 이미지 크기 조정
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 이미지 정규화
    ToTensorV2(),  # 텐서로 변환
])

original_transform = A.Compose([
    A.Resize(256, 256),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# 데이터셋 및 데이터로더 생성
train_dataset = AutonomousDrivingDataset(root_dir='training', original_transform=original_transform, augment_transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)

# 모델 정의 및 수정
model = deeplabv3_mobilenet_v3_large(pretrained=False, num_classes=25)

# 손실 함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)

# 혼합 정밀도 학습 스케일러 정의
scaler = GradScaler()

# 학습 함수
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for images, masks in dataloader:
        images, masks = images.to(device), masks.to(device)
        
        optimizer.zero_grad()
        with autocast():
            outputs = model(images)['out']
            loss = criterion(outputs, masks)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        total_loss += loss.item()
    
    return total_loss / len(dataloader)

# 검증 함수
def validate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for images, masks in dataloader:
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)['out']
            loss = criterion(outputs, masks)
            total_loss += loss.item()
    return total_loss / len(dataloader)

# Early stopping 클래스
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

# 학습 실행
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 24
early_stopping = EarlyStopping(patience=5, min_delta=0.001)
best_val_loss = float('inf')

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    val_loss = validate(model, train_loader, criterion, device)  # 실제로는 별도의 검증 데이터셋을 사용해야 합니다
    
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
    scheduler.step()  # 학습률 스케줄러 업데이트

    # 최상의 모델 저장
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_autonomous_driving_segmentation_model.pth')
        print(f"Saved best model at epoch {epoch+1}")
    
    # Early stopping 체크
    early_stopping(val_loss)
    if early_stopping.early_stop:
        print(f"Early stopping at epoch {epoch+1}")
        break
    
    # 3 에폭마다 모델 저장
    if (epoch + 1) % 3 == 0:
        torch.save(model.state_dict(), f'autonomous_driving_segmentation_model_epoch_{epoch+1}.pth')
        print(f"Saved model at epoch {epoch+1}")

# 최종 모델 저장
torch.save(model.state_dict(), 'final_autonomous_driving_segmentation_model.pth')
print("Training complete. Final model saved.")

# 모델을 양자화하고 저장
model.eval()
model.qconfig = torch.quantization.get_default_qconfig('fbgemm')
torch.quantization.prepare(model, inplace=True)

# 양자화된 모델을 학습 데이터로 프리-트레이닝
with torch.no_grad():
    for images, masks in train_loader:
        images, masks = images.to(device), masks.to(device)
        outputs = model(images)['out']

torch.quantization.convert(model, inplace=True)
torch.save(model.state_dict(), 'quantized_model.pth')
print("Quantized model saved.")


/home/mira/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/mira/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipykernel_3478584/2830687370.py:111: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/home/mira/anaconda3/lib/python3.12/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/tmp/ipykernel_3478584/2830687370.py:121: FutureWarning: `torch.cuda.amp.autocast(args...)`